In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [0]:
import numpy as np
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words = 10000)

2113536/2110848 [==============================] - 2s 1us/step


ValueError: ignored

In [0]:
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

# restore np.load for future normal usage
np.load = np_load_old

In [0]:
len(train_data)

8982

In [0]:
len(test_data)

2246

In [0]:
train_data[0]

[1,
 2,
 2,
 8,
 43,
 10,
 447,
 5,
 25,
 207,
 270,
 5,
 3095,
 111,
 16,
 369,
 186,
 90,
 67,
 7,
 89,
 5,
 19,
 102,
 6,
 19,
 124,
 15,
 90,
 67,
 84,
 22,
 482,
 26,
 7,
 48,
 4,
 49,
 8,
 864,
 39,
 209,
 154,
 6,
 151,
 6,
 83,
 11,
 15,
 22,
 155,
 11,
 15,
 7,
 48,
 9,
 4579,
 1005,
 504,
 6,
 258,
 6,
 272,
 11,
 15,
 22,
 134,
 44,
 11,
 15,
 16,
 8,
 197,
 1245,
 90,
 67,
 52,
 29,
 209,
 30,
 32,
 132,
 6,
 109,
 15,
 17,
 12]

In [0]:
# 단어 디코딩해서 어떤뉴스인지/for로 키 밸류값 가져와서 스페이스 넣고
word_index = reuters.get_word_index()
reverse = dict([(value, key) for (key, value) in word_index.items()])
decoded = ' '.join([reverse.get(i-3,'#') for i in train_data[0]]) 

In [0]:
decoded

'# # # said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

In [0]:
train_labels[0] # 3번 카테고리

3

In [0]:
import numpy as np

def vectorize_sequences(sequences, dimension = 10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results

In [0]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data) 

In [0]:
# 원핫인코딩 필요 0~46인덱스에서 하나만 1 // 8000몇*46의 테이블 생성
def to_one_hot(labels, dimension=46):
  results = np.zeros((len(labels), dimension))
  for i, label in enumerate(labels):
    results[i, label] = 1
  return results

In [0]:
y_train = to_one_hot(train_labels)
y_test = to_one_hot(test_labels)

In [0]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)
# 케라스 to_categorical사용하여 원핫인코딩

In [0]:
y_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [0]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape = (10000,)))
model.add(layers.Dense(128, activation='relu')) # 64차원의 데이터를 4차원으로 축소(정보소실), 축소한걸 다시 46으로 뺴니까 정보손실 큼 -> 정확도 떨어짐/차라리 여러단계로 조금씩 낮추면 ㄱㅊ
model.add(layers.Dense(46, activation='softmax'))

In [0]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
# 검증데이터 2000개
x_val = x_train[:2000]
partial_x_train = x_train[2000:] # 검증데이터와 비교할 train데이터

y_val = y_train[:2000]
partial_y_train = y_train[2000:]

In [0]:
history = model.fit(partial_x_train, partial_y_train, epochs = 12, batch_size = 512, validation_data = (x_val, y_val))

Train on 6982 samples, validate on 2000 samples
Epoch 1/12
6982/6982 [==============================] - 1s 98us/step - loss: 0.1154 - acc: 0.9608 - val_loss: 0.1501 - val_acc: 0.9545
Epoch 2/12
6982/6982 [==============================] - 1s 77us/step - loss: 0.1034 - acc: 0.9618 - val_loss: 0.1912 - val_acc: 0.9435
Epoch 3/12
6982/6982 [==============================] - 1s 83us/step - loss: 0.0972 - acc: 0.9659 - val_loss: 0.2236 - val_acc: 0.9420
Epoch 4/12
6982/6982 [==============================] - 1s 84us/step - loss: 0.0924 - acc: 0.9628 - val_loss: 0.2310 - val_acc: 0.9425
Epoch 5/12
6982/6982 [==============================] - 1s 76us/step - loss: 0.0933 - acc: 0.9675 - val_loss: 0.2504 - val_acc: 0.9425
Epoch 6/12
6982/6982 [==============================] - 1s 75us/step - loss: 0.0956 - acc: 0.9635 - val_loss: 0.2560 - val_acc: 0.9420
Epoch 7/12
6982/6982 [==============================] - 1s 74us/step - loss: 0.0867 - acc: 0.9655 - val_loss: 0.2643 - val_acc: 0.9400
Epoch 8

In [0]:
model.fit(x_train, y_train, epochs = 12, batch_size = 512)

Epoch 1/12
8982/8982 [==============================] - 1s 62us/step - loss: 0.1271 - acc: 0.9595
Epoch 2/12
8982/8982 [==============================] - 1s 62us/step - loss: 0.1014 - acc: 0.9598
Epoch 3/12
8982/8982 [==============================] - 1s 62us/step - loss: 0.1057 - acc: 0.9598
Epoch 4/12
8982/8982 [==============================] - 1s 63us/step - loss: 0.1079 - acc: 0.9576
Epoch 5/12
8982/8982 [==============================] - 1s 61us/step - loss: 0.0988 - acc: 0.9588
Epoch 6/12
8982/8982 [==============================] - 1s 60us/step - loss: 0.0991 - acc: 0.9571
Epoch 7/12
8982/8982 [==============================] - 1s 60us/step - loss: 0.0992 - acc: 0.9582
Epoch 8/12
8982/8982 [==============================] - 1s 60us/step - loss: 0.0959 - acc: 0.9585
Epoch 9/12
8982/8982 [==============================] - 1s 60us/step - loss: 0.0902 - acc: 0.9590
Epoch 10/12
8982/8982 [==============================] - 1s 60us/step - loss: 0.0966 - acc: 0.9579
Epoch 11/12
8982/89

In [0]:
model.evaluate(x_test, y_test)

2246/2246 [==============================] - 0s 90us/step


[1.1184056191601504, 0.8014247551732899]

In [0]:
predictions = model.predict(x_test)

In [0]:
predictions[0]

array([2.3468629e-06, 4.7992888e-08, 3.5914544e-10, 9.9584097e-01,
       4.1283388e-03, 1.8613405e-09, 7.9796134e-09, 1.7784163e-08,
       5.0334938e-06, 1.4370889e-07, 3.3855000e-07, 2.1579979e-06,
       4.5981361e-09, 1.8177174e-06, 2.5365630e-09, 2.6914060e-09,
       1.6187431e-06, 1.3590522e-07, 4.3948334e-09, 7.2527109e-06,
       2.5182940e-06, 3.7690601e-08, 1.2147424e-08, 6.0077959e-08,
       1.2697997e-11, 3.3655205e-08, 7.5064799e-10, 3.7300836e-08,
       2.9814643e-08, 4.7605531e-07, 1.9490264e-07, 7.1219364e-09,
       1.6966185e-07, 4.8188968e-09, 3.2715681e-09, 2.0672589e-10,
       3.6248705e-06, 9.7429528e-08, 4.5991559e-08, 2.2394988e-06,
       6.1482353e-09, 2.1286944e-10, 2.0518844e-09, 8.2044359e-08,
       1.9621389e-09, 1.0298923e-09], dtype=float32)

In [0]:
np.sum(predictions[0])

0.99999994

In [0]:
np.argmax(predictions[0])

3